## Dataset


In [17]:
lyrics1 = [
    ("Cause I-I-I'm in the stars tonight", 0.9),
    ("So watch me bring the fire and set the night alight", 0.8),
    ("Shining through the city with a little funk and soul", 0.7),
    ("So I'ma light it up like dynamite", 0.6)
    ]

lyrics2 = [
    ("Cause I-I-I'm in the stars tonight", 0.5),
    ("So watch me bring the fire and set the night alight", 0.4),
    ("Bring a friend, join the crowd", 0.3),
    ("Just move like we off the wall", 0.2)
    ]

## RRF

In [ ]:
from typing import List, Tuple


def rrf_combination(
    lyrics1: List[Tuple[str, float]],
    lyrics2: List[Tuple[str, float]],
    k: int = 60 # RRF에서 사용하는 상수 (기본값 60)
) -> List[Tuple[str, float]]:

    combined_results = {}
    
    # 첫 번째 결과의 Reciprocal Rank 계산 및 점수 합산
    for rank, (text, score) in enumerate(lyrics1):
        if text not in combined_results:
            combined_results[text] = 0
        combined_results[text] += score + 1 / (k + rank)
    
    # 두 번째 결과의 Reciprocal Rank 계산 및 점수 합산
    for rank, (text, score) in enumerate(lyrics2):
        if text not in combined_results:
            combined_results[text] = 0
        combined_results[text] += score + 1 / (k + rank)
    
    # 사전의 값을 리스트로 변환하고, 점수에 따라 내림차순으로 정렬
    sorted_combined_results = sorted(combined_results.items(), key=lambda x: x[1], reverse=True)

    return sorted_combined_results

In [45]:
rrf_combination(lyrics1, lyrics2)

[("Cause I-I-I'm in the stars tonight", 1.4333333333333336),
 ('So watch me bring the fire and set the night alight', 1.2327868852459019),
 ('Shining through the city with a little funk and soul', 0.7161290322580645),
 ("So I'ma light it up like dynamite", 0.6158730158730159),
 ('Bring a friend, join the crowd', 0.3161290322580645),
 ('Just move like we off the wall', 0.21587301587301588)]

## Linear Combination

In [18]:
# None 값을 0으로 치환하는 함수
def safe_float(value):
    return 0.0 if value is None else float(value)

In [ ]:
import math


def normalize_score(score, min_score, max_score):
    """
    점수를 정규화합니다.
    1. Min-Max 정규화를 통해 입력 점수를 0-1 범위로 조정
    2. 로그 변환을 적용하여 점수 분포 조정
    3. 지수 함수를 사용하여 최종 유사도 점수 계산

    Parameters:
    rank (float): 변환할 개별 문서의 점수
    min_rank (float): 현재 검색 결과 세트의 최소 점수
    max_rank (float): 현재 검색 결과 세트의 최대 점수

    Returns:
    float: 0-1 사이의 점수
    """
    # 1. Min-Max 정규화
    if (score - min_score) == 0 and (max_score - min_score) == 0:
        normalized_rank = 0
    else:
        normalized_rank = (score - min_score) / (max_score - min_score)
    
    # 2. 로그 변환 (1을 더해 0을 방지)
    log_rank = math.log(1 + normalized_rank)
    
    # 3. 지수 변환으로 0-1 범위로 매핑
    cosine_like_similarity = 1 - math.exp(-log_rank)
    
    return cosine_like_similarity

In [ ]:
from typing import List, Tuple

def linear_combination_search_results(
    lyrics1: List[Tuple[str, float]],
    lyrics2: List[Tuple[str, float]],
    alpha: float = 0.3, # 두 결과 간의 가중치를 조정하는 파라미터
) ->List[Tuple[str, float]]:

    # 결과를 결합하기 위한 사전 생성
    combined_results = {}
    
    # 첫 번째 결과 추가
    for (text, score) in lyrics1:
        if text not in combined_results:
            combined_results[text] = alpha * safe_float(score)
        else:
            existing_score = combined_results[text]
            combined_results[text] = safe_float(existing_score) + alpha * safe_float(score)

    # 두번째 결과 추가
    for (text, score) in lyrics2:
        if text not in combined_results:
            combined_results[text] = alpha * safe_float(score)
        else:
            existing_score = combined_results[text]
            combined_results[text] = safe_float(existing_score) + alpha * safe_float(score)
            
    # 결과 점수 추출
    scores = list(combined_results.values())
    
    # 최소값과 최대값 계산
    min_score = min(scores)
    max_score = max(scores)
    
    # 점수 정규화 및 정렬
    normalized_results = [(text, normalize_score(score, min_score, max_score)) for text, score in combined_results.items()]

    # 사전의 값을 리스트로 변환하고, 점수에 따라 내림차순으로 정렬
    normalized_results.sort(key=lambda x: x[1], reverse=True)

    return normalized_results


In [30]:
linear_combination_search_results(lyrics1, lyrics2)

[("Cause I-I-I'm in the stars tonight", 0.5),
 ('So watch me bring the fire and set the night alight', 0.4545454545454545),
 ('Shining through the city with a little funk and soul', 0.2941176470588235),
 ("So I'ma light it up like dynamite", 0.2499999999999999),
 ('Bring a friend, join the crowd', 0.07692307692307687),
 ('Just move like we off the wall', 0.0)]

## Borda Count

In [ ]:
def borda_count_combination(
    lyrics1: List[Tuple[str, float]],
    lyrics2: List[Tuple[str, float]],
) ->List[Tuple[str, float]]:

    combined_results = {}
    
    # 첫 번째 결과에 순위 점수 부여
    for rank, (text, score) in enumerate(lyrics1):
        if text not in combined_results:
            combined_results[text] = 0
        combined_results[text] =  combined_results[text] + len(lyrics1) - rank
    
    # 두 번째 결과에 순위 점수 부여
    for rank, (text, score) in enumerate(lyrics2):
        if text not in combined_results:
            combined_results[text] = 0
        combined_results[text] =  combined_results[text] + len(lyrics2) - rank
    

    # 사전의 값을 리스트로 변환하고, 점수에 따라 내림차순으로 정렬
    sorted_combined_results = sorted(combined_results.items(), key=lambda x: x[1], reverse=True)

    return sorted_combined_results

In [39]:
borda_count_combination(lyrics1, lyrics2)

[("Cause I-I-I'm in the stars tonight", 8),
 ('So watch me bring the fire and set the night alight', 6),
 ('Shining through the city with a little funk and soul', 2),
 ('Bring a friend, join the crowd', 2),
 ("So I'ma light it up like dynamite", 1),
 ('Just move like we off the wall', 1)]